本秘籍提供了使用PyTorch基准测试模块测量和比较代码性能的快速入门指南。

# 介绍

基准测试是写代码的重要步骤。它是验证我们的代码是否满足预期的性能方法，比较解决同一问题的不同并防止回归性能。

在对 Pyorch 代码进行基准测试时有很多选择，包括 Python 内置的时间它模块。但是，对 PyTorch 代码进行基准测试有很多容易被注意事项，管理线程数量和同步 CUDA 设备。另外，为基准测试 生成 Tensor 输入可能非常乏味。

这个秘籍演示了如何使用 PyTorch 基准测试模块来避免常见错误，同时更容易比较不同代码的性能、生成基准测试输入等。

# 步骤
1. 定义函数以进行基准测试
2. 使用 timeit.Timer 进行基准测试
3. 使用 torch.utils.benchmark.Timer 进行基准测试
4. 使用阻塞式自动量程进行基准测试
5. 比较基准结果
6. 保存/加载基准测试结果
7. 使用模糊参数生成输入
8. 使用 Callgrind 收集指令计数

1. 定义函数以进行基准测试

在撰写本文时，torch.dot 不支持批处理模式，因此我们将比较使用现有 Torch 运算符实现它的两种方法：一种方法使用 mul 和 sum 的组合，而另一种方法将问题减少到 bmm。

In [1]:
import torch


def batched_dot_mul_sum(a, b):
    '''Computes batched dot by multiplying and summing'''
    return a.mul(b).sum(-1)


def batched_dot_bmm(a, b):
    '''Computes batched dot by reducing to bmm'''
    a = a.reshape(-1, 1, a.shape[-1])
    b = b.reshape(-1, b.shape[-1], 1)
    return torch.bmm(a, b).flatten(-3)


# Input for benchmarking
x = torch.randn(10000, 64)

# Ensure that both functions compute the same output
assert batched_dot_mul_sum(x, x).allclose(batched_dot_bmm(x, x))

2. 使用 timeit.Timer 进行基准测试

首先，让我们使用 Python 的内置 timeit 模块对代码进行基准测试。 我们在这里保持基准代码简单，以便我们可以比较 timeit 和 torch.utils.benchmark 的默认值。

In [2]:
import timeit

t0 = timeit.Timer(
    stmt='batched_dot_mul_sum(x, x)',
    setup='from __main__ import batched_dot_mul_sum',
    globals={'x': x})

t1 = timeit.Timer(
    stmt='batched_dot_bmm(x, x)',
    setup='from __main__ import batched_dot_bmm',
    globals={'x': x})

print(f'mul_sum(x, x):  {t0.timeit(100) / 100 * 1e6:>5.1f} us')
print(f'bmm(x, x):      {t1.timeit(100) / 100 * 1e6:>5.1f} us')

mul_sum(x, x):  123.0 us
bmm(x, x):      122.9 us


3. 使用 torch.utils.benchmark.Timer 进行基准测试

PyTorch 基准测试模块旨在让之前使用过 timeit 模块的人熟悉。 但是，它的默认设置使得用于对 PyTorch 代码进行基准测试更容易、更安全。 让我们首先比较与上面相同的基本 API。

In [3]:
import torch.utils.benchmark as benchmark

t0 = benchmark.Timer(
    stmt='batched_dot_mul_sum(x, x)',
    setup='from __main__ import batched_dot_mul_sum',
    globals={'x': x})

t1 = benchmark.Timer(
    stmt='batched_dot_bmm(x, x)',
    setup='from __main__ import batched_dot_bmm',
    globals={'x': x})

print(t0.timeit(100))
print(t1.timeit(100))

batched_dot_mul_sum(x, x)
setup: from __main__ import batched_dot_mul_sum
  409.42 us
  1 measurement, 100 runs , 1 thread
batched_dot_bmm(x, x)
setup: from __main__ import batched_dot_bmm
  764.43 us
  1 measurement, 100 runs , 1 thread


尽管 API 的基本功能相同，但还是存在一些重要差异。 benchmark.Timer.timeit() 返回每次运行的时间，而不是像 timeit.Timer.timeit() 那样的总运行时间。 PyTorch 基准模块还提供用于打印结果的格式化字符串表示。

另一个重要的区别，也是结果不同的原因是 PyTorch 基准测试模块默认在单线程中运行。 我们可以使用 num_threads 参数更改线程数。

torch.utils.benchmark.Timer 需要几个额外的参数，包括：label、sub_label、description 和 env，它们改变了返回的测量对象的 __repr__ 并用于对结果进行分组（稍后会详细介绍）。

In [4]:
num_threads = torch.get_num_threads()
print(f'Benchmarking on {num_threads} threads')

t0 = benchmark.Timer(
    stmt='batched_dot_mul_sum(x, x)',
    setup='from __main__ import batched_dot_mul_sum',
    globals={'x': x},
    num_threads=num_threads,
    label='Multithreaded batch dot',
    sub_label='Implemented using mul and sum')

t1 = benchmark.Timer(
    stmt='batched_dot_bmm(x, x)',
    setup='from __main__ import batched_dot_bmm',
    globals={'x': x},
    num_threads=num_threads,
    label='Multithreaded batch dot',
    sub_label='Implemented using bmm')

print(t0.timeit(100))
print(t1.timeit(100))

Benchmarking on 20 threads
Multithreaded batch dot: Implemented using mul and sum
setup: from __main__ import batched_dot_mul_sum
  84.94 us
  1 measurement, 100 runs , 20 threads
Multithreaded batch dot: Implemented using bmm
setup: from __main__ import batched_dot_bmm
  107.02 us
  1 measurement, 100 runs , 20 threads


使用所有可用线程运行基准测试给出与 timeit 模块类似的结果。 更重要的是，哪个版本更快取决于我们运行代码的线程数。 这就是为什么使用代表实际用例的线程设置对代码进行基准测试很重要的原因。 要记住的另一件重要事情是在 GPU 上进行基准测试时同步 CPU 和 CUDA。 让我们在 CUDA 张量上再次运行上述基准测试，看看会发生什么。

In [5]:
x = torch.randn(10000, 1024, device='cuda:0')

t0 = timeit.Timer(
    stmt='batched_dot_mul_sum(x, x)',
    setup='from __main__ import batched_dot_mul_sum',
    globals={'x': x})

t1 = timeit.Timer(
    stmt='batched_dot_bmm(x, x)',
    setup='from __main__ import batched_dot_bmm',
    globals={'x': x})

# Ran each twice to show difference before/after warmup
print(f'mul_sum(x, x):  {t0.timeit(100) / 100 * 1e6:>5.1f} us')
print(f'mul_sum(x, x):  {t0.timeit(100) / 100 * 1e6:>5.1f} us')
print(f'bmm(x, x):      {t1.timeit(100) / 100 * 1e6:>5.1f} us')
print(f'bmm(x, x):      {t1.timeit(100) / 100 * 1e6:>5.1f} us')

mul_sum(x, x):   40.3 us
mul_sum(x, x):   36.2 us
bmm(x, x):      985.3 us
bmm(x, x):       45.8 us


In [6]:
t0 = benchmark.Timer(
    stmt='batched_dot_mul_sum(x, x)',
    setup='from __main__ import batched_dot_mul_sum',
    globals={'x': x})

t1 = benchmark.Timer(
    stmt='batched_dot_bmm(x, x)',
    setup='from __main__ import batched_dot_bmm',
    globals={'x': x})

# Run only once since benchmark module does warmup for us
print(t0.timeit(100))
print(t1.timeit(100))

batched_dot_mul_sum(x, x)
setup: from __main__ import batched_dot_mul_sum
  350.22 us
  1 measurement, 100 runs , 1 thread
batched_dot_bmm(x, x)
setup: from __main__ import batched_dot_bmm
  346.89 us
  1 measurement, 100 runs , 1 thread


结果揭示了一些有趣的事情。 使用 timeit 模块的 bmm 版本的第一次运行比第二次运行要长得多。 这是因为 bmm 调用了 cuBLAS，它需要在第一次调用时加载，这需要一些时间。 这就是为什么在基准测试之前进行热身运行很重要的原因，对我们来说幸运的是，PyTorch 的基准测试模块会处理这个问题。

timeit 和 benchmark 模块之间的结果差异是因为 timeit 模块不同步 CUDA，因此只计时启动内核的时间。 PyTorch 的 benchmark 模块为我们做同步。

4. 使用阻塞式自动量程进行基准测试

虽然 timeit.Timer.autorange 进行至少 0.2 秒的单次连续测量，但 torch.utils.benchmark.blocked_autorange 进行多次测量，其时间总计至少为 0.2 秒（可以通过 min_run_time 参数更改）受时间限制 开销只是整体测量的一小部分。 这是通过首先在每个循环中增加运行次数来实现的，直到运行时间远大于测量开销（这也用作预热），然后进行测量直到达到目标时间。 这具有有用的特性，即浪费更少的数据，并允许我们计算统计数据来估计测量的可靠性。

In [7]:
m0 = t0.blocked_autorange()
m1 = t1.blocked_autorange()

print(m0)
print(m1)

batched_dot_mul_sum(x, x)
setup: from __main__ import batched_dot_mul_sum
  327.98 us
  1 measurement, 1000 runs , 1 thread
batched_dot_bmm(x, x)
setup: from __main__ import batched_dot_bmm
  315.80 us
  1 measurement, 1000 runs , 1 thread


我们还可以从返回的测量对象中检查单个统计信息。

In [8]:
print(f"Mean:   {m0.mean * 1e6:6.2f} us")
print(f"Median: {m0.median * 1e6:6.2f} us")

Mean:   327.98 us
Median: 327.98 us


5. 比较基准结果

到目前为止，我们一直在将两个版本的批处理点与单个输入进行比较。 在实践中，我们想尝试输入的组合以及不同数量的线程。 Compare 类有助于在格式化的表格中显示许多测量的结果。 它使用上面描述的注释（label、sub_label、num_threads 等）以及 description 来对表进行分组和组织。 让我们使用 Compare 来看看我们的函数在不同的输入大小和线程数下的表现如何。

In [9]:
from itertools import product

# Compare takes a list of measurements which we'll save in results.
results = []

sizes = [1, 64, 1024, 10000]
for b, n in product(sizes, sizes):
    # label and sub_label are the rows
    # description is the column
    label = 'Batched dot'
    sub_label = f'[{b}, {n}]'
    x = torch.ones((b, n))
    for num_threads in [1, 4, 16, 32]:
        results.append(benchmark.Timer(
            stmt='batched_dot_mul_sum(x, x)',
            setup='from __main__ import batched_dot_mul_sum',
            globals={'x': x},
            num_threads=num_threads,
            label=label,
            sub_label=sub_label,
            description='mul/sum',
        ).blocked_autorange(min_run_time=1))
        results.append(benchmark.Timer(
            stmt='batched_dot_bmm(x, x)',
            setup='from __main__ import batched_dot_bmm',
            globals={'x': x},
            num_threads=num_threads,
            label=label,
            sub_label=sub_label,
            description='bmm',
        ).blocked_autorange(min_run_time=1))

compare = benchmark.Compare(results)
compare.print()

[--------------- Batched dot ----------------]
                      |  mul/sum   |    bmm   
1 threads: -----------------------------------
      [1, 1]          |       8.5  |      12.6
      [1, 64]         |       4.8  |       7.5
      [1, 1024]       |       5.1  |       8.5
      [1, 10000]      |       6.7  |       9.3
      [64, 1]         |       4.9  |       8.4
      [64, 64]        |       6.7  |      12.7
      [64, 1024]      |      26.8  |     164.6
      [64, 10000]     |     341.5  |    1478.8
      [1024, 1]       |       6.1  |      14.9
      [1024, 64]      |      34.7  |      86.4
      [1024, 1024]    |     531.7  |    2420.1
      [1024, 10000]   |   23665.7  |   23536.0
      [10000, 1]      |      13.6  |      74.9
      [10000, 64]     |     388.0  |     722.1
      [10000, 1024]   |   23451.2  |   23552.1
      [10000, 10000]  |  238468.1  |  228073.2
4 threads: -----------------------------------
      [1, 1]          |       8.5  |      12.6
      [1, 64]

上面的结果表明，对于在多线程上运行的较大张量，减少到 bmm 的版本更好，而对于较小和/或单线程代码，另一个版本更好。

比较还提供了改变表格格式的功能

In [10]:
compare.trim_significant_figures()
compare.colorize()
compare.print()

[-------------- Batched dot --------------]
                      |  mul/sum  |   bmm  
1 threads: --------------------------------
      [1, 1]          |        8  |      13
      [1, 64]         |        5  |       8
      [1, 1024]       |        5  |       8
      [1, 10000]      |        7  |       9
      [64, 1]         |        5  |       8
      [64, 64]        |        7  |      13
      [64, 1024]      |       27  |     160
      [64, 10000]     |      340  |    1500
      [1024, 1]       |        6  |      15
      [1024, 64]      |       35  |      86
      [1024, 1024]    |      532  |    2400
      [1024, 10000]   |    24000  |   24000
      [10000, 1]      |       14  |      75
      [10000, 64]     |      390  |     720
      [10000, 1024]   |    23000  |   23600
      [10000, 10000]  |   200000  |  230000
4 threads: --------------------------------
      [1, 1]          |        8  |      13
      [1, 64]         |        5  |       8
      [1, 1024]       |        5

6. 保存/加载基准测试结果

测量值（和第 8 节中描述的 CallgrindStats）是可以选择的。 这使得 A/B 测试变得容易，因为您可以从两个不同的环境中收集测量值，对它们进行腌制，然后将它们加载到单个环境中。 Timer 甚至采用 env 构造函数参数，以便此类 A/B 测试无缝工作。

让我们想象一下，不是两个 Python 函数，而是 add/sum 和 bmm 方法在 PyTorch 的两个不同版本中。 下面的示例演示了如何对它们进行 A/B 测试。 为简单起见，我们只使用形状的子集，并通过 pickle 简单地往返结果，而不是实际使用多个环境并将结果写入磁盘。

In [11]:
import pickle

ab_test_results = []
for env in ('environment A: mul/sum', 'environment B: bmm'):
    for b, n in ((1, 1), (1024, 10000), (10000, 1)):
        x = torch.ones((b, n))
        dot_fn = (batched_dot_mul_sum if env == 'environment A: mul/sum' else batched_dot_bmm)
        m = benchmark.Timer(
            stmt='batched_dot(x, x)',
            globals={'x': x, 'batched_dot': dot_fn},
            num_threads=1,
            label='Batched dot',
            description=f'[{b}, {n}]',
            env=env,
        ).blocked_autorange(min_run_time=1)
        ab_test_results.append(pickle.dumps(m))

ab_results = [pickle.loads(i) for i in ab_test_results]
compare = benchmark.Compare(ab_results)
compare.trim_significant_figures()
compare.colorize()
compare.print()

[------------------------------------- Batched dot -------------------------------------]
                                               |  [1, 1]  |  [1024, 10000]  |  [10000, 1]
1 threads: ------------------------------------------------------------------------------
  (environment A: mul/sum)  batched_dot(x, x)  |   4.7    |      24000      |      14    
  (environment B: bmm)      batched_dot(x, x)  |   7.4    |      24000      |      75    

Times are in microseconds (us).



In [12]:
# And just to show that we can round trip all of the results from earlier:
round_tripped_results = pickle.loads(pickle.dumps(results))
assert(str(benchmark.Compare(results)) == str(benchmark.Compare(round_tripped_results)))

7. 使用模糊参数生成输入

正如我们在上一节中看到的，根据输入张量的不同，可能会有一些明显的性能差异。 因此，在许多不同的输入上运行基准测试是一个好主意。 但是，创建所有这些输入张量可能很乏味，这就是 torch.utils.benchmark.Fuzzer 和相关类的用武之地。让我们看看如何使用 Fuzzer 为基准测试创建一些测试用例。

In [13]:
from torch.utils.benchmark import Fuzzer, FuzzedParameter, FuzzedTensor, ParameterAlias

# Generates random tensors with 128 to 10000000 elements and sizes k0 and k1 chosen from a
# loguniform distribution in [1, 10000], 40% of which will be discontiguous on average.
example_fuzzer = Fuzzer(
    parameters = [
        FuzzedParameter('k0', minval=1, maxval=10000, distribution='loguniform'),
        FuzzedParameter('k1', minval=1, maxval=10000, distribution='loguniform'),
    ],
    tensors = [
        FuzzedTensor('x', size=('k0', 'k1'), min_elements=128, max_elements=10000000, probability_contiguous=0.6)
    ],
    seed=0,
)

results = []
for tensors, tensor_params, params in example_fuzzer.take(10):
    # description is the column label
    sub_label=f"{params['k0']:<6} x {params['k1']:<4} {'' if tensor_params['x']['is_contiguous'] else '(discontiguous)'}"
    results.append(benchmark.Timer(
        stmt='batched_dot_mul_sum(x, x)',
        setup='from __main__ import batched_dot_mul_sum',
        globals=tensors,
        label='Batched dot',
        sub_label=sub_label,
        description='mul/sum',
    ).blocked_autorange(min_run_time=1))
    results.append(benchmark.Timer(
        stmt='batched_dot_bmm(x, x)',
        setup='from __main__ import batched_dot_bmm',
        globals=tensors,
        label='Batched dot',
        sub_label=sub_label,
        description='bmm',
    ).blocked_autorange(min_run_time=1))

compare = benchmark.Compare(results)
compare.trim_significant_figures()
compare.print()

[--------------------- Batched dot ---------------------]
                                     |  mul/sum  |   bmm 
1 threads: ----------------------------------------------
      725    x 257                   |     100   |    248
      49     x 383                   |      15   |     38
      34     x 1468                  |      19   |    130
      187    x 5039                  |     550   |   2200
      2140   x 1296 (discontiguous)  |    2000   |  30100
      78     x 1598                  |      46   |    304
      519    x 763                   |     240   |    956
      141    x 1082                  |      67   |    380
      78     x 5    (discontiguous)  |      10   |     15
      187    x 1                     |       9   |     15

Times are in microseconds (us).



定义自己的 Fuzzer 有很大的灵活性，这对于创建一组强大的基准测试输入非常有用。 但为了让事情变得更简单，PyTorch 基准测试模块附带了一些用于常见基准测试需求的模糊器。 让我们来看看如何使用这些内置模糊器之一。

In [14]:
from torch.utils.benchmark.op_fuzzers import binary

results = []
for tensors, tensor_params, params in binary.BinaryOpFuzzer(seed=0).take(10):
    sub_label=f"{params['k0']:<6} x {params['k1']:<4} {'' if tensor_params['x']['is_contiguous'] else '(discontiguous)'}"
    results.append(benchmark.Timer(
        stmt='batched_dot_mul_sum(x, x)',
        setup='from __main__ import batched_dot_mul_sum',
        globals=tensors,
        label='Batched dot',
        sub_label=sub_label,
        description='mul/sum',
    ).blocked_autorange(min_run_time=1))
    results.append(benchmark.Timer(
        stmt='batched_dot_bmm(x, x)',
        setup='from __main__ import batched_dot_bmm',
        globals=tensors,
        label='Batched dot',
        sub_label=sub_label,
        description='bmm',
    ).blocked_autorange(min_run_time=1))

compare = benchmark.Compare(results)
compare.trim_significant_figures()
compare.colorize(rowwise=True)
compare.print()

[----------------------- Batched dot ------------------------]
                                         |  mul/sum  |   bmm  
1 threads: ---------------------------------------------------
      64     x 473  (discontiguous)      |   26500   |   96000
      16384  x 12642115 (discontiguous)  |      23   |      86
      8192   x 892                       |    6800   |   17100
      512    x 64   (discontiguous)      |   92000   |  300000
      493    x 27   (discontiguous)      |    2300   |    4900
      118    x 32   (discontiguous)      |    1050   |    3500
      16     x 495  (discontiguous)      |   24000   |   25100
      488    x 62374                     |   70000   |   70000
      240372 x 69                        |   40000   |   19000
      40156  x 32   (discontiguous)      |    2100   |    4600

Times are in microseconds (us).



8. 使用 Callgrind 收集指令计数

优化代码的挑战之一是挂墙时间的变化和不透明度。不确定性的来源有很多，从自适应时钟速度到与其他进程的资源争用。此外，端到端时间无法洞察时间花在哪里，而这正是我们在优化代码时真正感兴趣的。

一种补充方法是还收集指令计数。这些计数是一个代理指标，并没有捕获性能的所有方面（例如内存或 I/O 绑定任务），但它们确实有几个有用的属性。指令计数是可重复的，对环境变化不敏感，并提供对程序在哪里花费周期的细粒度洞察。

要了解指令计数的效用，让我们看看如何减少 batched_dot_mul_sum 的开销。显而易见的解决方案是将其移至 C++，因此我们避免在 Python 和 C++ 之间多次切换。

幸运的是，来源几乎相同。我们在 C++ 中必须问的一个问题是我们应该按值还是按引用来获取参数。

In [15]:
batched_dot_src = """\
/* ---- Python ---- */
// def batched_dot_mul_sum(a, b):
//     return a.mul(b).sum(-1)

torch::Tensor batched_dot_mul_sum_v0(
    const torch::Tensor a,
    const torch::Tensor b) {
  return a.mul(b).sum(-1);
}

torch::Tensor batched_dot_mul_sum_v1(
    const torch::Tensor& a,
    const torch::Tensor& b) {
  return a.mul(b).sum(-1);
}
"""


# PyTorch makes it easy to test our C++ implementations by providing a utility
# to JIT compile C++ source into Python extensions:
import os
from torch.utils import cpp_extension
cpp_lib = cpp_extension.load_inline(
    name='cpp_lib',
    cpp_sources=batched_dot_src,
    extra_cflags=['-O3'],
    extra_include_paths=[
        # `load_inline` needs to know where to find Pybind11 headers.
        os.path.join(os.getenv('CONDA_PREFIX'), 'include')
    ],
    functions=['batched_dot_mul_sum_v0', 'batched_dot_mul_sum_v1']
)

# `load_inline` will create a shared object that is loaded into Python. When we collect
# instruction counts Timer will create a subprocess, so we need to re-import it. The
# import process is slightly more complicated for C extensions, but that's all we're
# doing here.
module_import_str = f"""\
# https://stackoverflow.com/questions/67631/how-to-import-a-module-given-the-full-path
import importlib.util
spec = importlib.util.spec_from_file_location("cpp_lib", {repr(cpp_lib.__file__)})
cpp_lib = importlib.util.module_from_spec(spec)
spec.loader.exec_module(cpp_lib)"""

import textwrap
def pretty_print(result):
    """Import machinery for cpp_lib.so can get repetitive to look at."""
    print(repr(result).replace(textwrap.indent(module_import_str, "  "), "  import cpp_lib"))


t_baseline = benchmark.Timer(
    stmt='batched_dot_mul_sum(x, x)',
    setup='''\
from __main__ import batched_dot_mul_sum
x = torch.randn(2, 2)''')

t0 = benchmark.Timer(
    stmt='cpp_lib.batched_dot_mul_sum_v0(x, x)',
    setup=f'''\
{module_import_str}
x = torch.randn(2, 2)''')

t1 = benchmark.Timer(
    stmt='cpp_lib.batched_dot_mul_sum_v1(x, x)',
    setup=f'''\
{module_import_str}
x = torch.randn(2, 2)''')

# Moving to C++ did indeed reduce overhead, but it's hard to tell which
# calling convention is more efficient. v1 (call with references) seems to
# be a bit faster, but it's within measurement error.
pretty_print(t_baseline.blocked_autorange())
pretty_print(t0.blocked_autorange())
pretty_print(t1.blocked_autorange())

batched_dot_mul_sum(x, x)
setup:
  from __main__ import batched_dot_mul_sum
  x = torch.randn(2, 2)

  4.74 us
  1 measurement, 100000 runs , 1 thread
cpp_lib.batched_dot_mul_sum_v0(x, x)
setup:
  import cpp_lib
  x = torch.randn(2, 2)

  3.89 us
  1 measurement, 100000 runs , 1 thread
cpp_lib.batched_dot_mul_sum_v1(x, x)
setup:
  import cpp_lib
  x = torch.randn(2, 2)

  3.91 us
  1 measurement, 100000 runs , 1 thread


In [16]:
# Let's use Callgrind to determine which is better.
stats_v0 = t0.collect_callgrind()
stats_v1 = t1.collect_callgrind()

pretty_print(stats_v0)
pretty_print(stats_v1)

# `.as_standardized` removes file names and some path prefixes, and makes
# it easier to read the function symbols.
stats_v0 = stats_v0.as_standardized()
stats_v1 = stats_v1.as_standardized()

# `.delta` diffs the instruction counts, and `.denoise` removes several
# functions in the Python interpreter that are known to have significant
# jitter.
delta = stats_v1.delta(stats_v0).denoise()

# `.transform` is a convenience API for transforming function names. It is
# useful for increasing cancelation when diff-ing instructions, as well as
# just generally improving readability.
replacements = (
    ("???:void pybind11", "pybind11"),
    ("batched_dot_mul_sum_v0", "batched_dot_mul_sum_v1"),
    ("at::Tensor, at::Tensor", "..."),
    ("at::Tensor const&, at::Tensor const&", "..."),
    ("auto torch::detail::wrap_pybind_function_impl_", "wrap_pybind_function_impl_"),
)
for before, after in replacements:
    delta = delta.transform(lambda l: l.replace(before, after))

# We can use print options to control how much of the function to display.
torch.set_printoptions(linewidth=160)

# Once parsed, the instruction counts make clear that passing `a` and `b`
# by reference is more efficient as it skips some c10::TensorImpl bookkeeping
# for the intermediate Tensors, and is also works better with PyBind11. This
# is consistent with our noisy wall time observations.
print(delta)

OSError: Missing: valgrind, callgrind_control, callgrind_annotate